## COMP 569 - Harpreet Kaur & Soltan Soltanli

<hr/>

### 1) The function which will generate Grid based on given inputs (Sizes)

In [318]:
import random

# --- This function generates dynamic grid with random values ---
# --- But to keep it simple, we will put static grid ---

def generate_grid(rows, cols):
    grid = [[1 if r == 0 or r == rows - 1 or c == 0 or c == cols - 1 else random.choice([0, 1])
             for c in range(cols)] for r in range(rows)]

    r_pos = (random.randint(1, rows - 2), random.randint(1, cols - 2))
    d_pos = r_pos
    while d_pos == r_pos:
        d_pos = (random.randint(1, rows - 2), random.randint(1, cols - 2))

    grid[r_pos[0]][r_pos[1]] = 'R'
    grid[d_pos[0]][d_pos[1]] = 'D'

    return grid

# Dynamic Grid
grid = generate_grid(20, 20)

# Static Grid (the one in presentation)
# grid = [
#     [1, 1, 1, 1, 1, 1, 1, 1, 1],
#     [1, 0, 0, 0, 1, 0, 0, 0, 1],
#     [1, 0, 0, 0, 1, 0, 0, 'R', 1],
#     [1, 0, 0, 0, 0, 0, 0, 0, 1],
#     [1, 0, 0, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 1, 1, 1, 1],
#     [1, 1, 0, 0, 0, 0, 0, 'D', 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 1]]

for row in grid:
    print(row)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1]
[1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
[1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 'D', 1, 1, 1]
[1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 'R', 0, 0, 0, 1, 1]
[1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]
[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1]
[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1]
[1, 0, 0, 1, 0, 1, 1

### 2) Movements

In [319]:
from collections import deque

directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

### 3) Getting Start and Goal coordinates

In [320]:
start_loc = None
goal_loc = None

rows, cols = len(grid), len(grid[0])

for i in range(rows):
    for j in range(cols):
        if grid[i][j] == 'R':
            start_loc = (i, j)
        if grid[i][j] == 'D':
            goal_loc = (i, j)

if not start_loc or not goal_loc:
    print("Invalid grid, no start or goal point found.")

print("Start (R):", start_loc)
print("Goal (D):", goal_loc)

Start (R): (6, 14)
Goal (D): (4, 16)


### 4) Check if the current location is the goal

In [321]:
def goal_test(current_loc, goal_loc):
    return current_loc == goal_loc

### 5) Successor function: Generates valid next moves (sliding and stepping)

In [322]:
def successor_fcn(grid, current_node):
    x, y = current_node['location']
    successors = []

    # Step movement
    for dx, dy in directions:
        nx, ny = x + dx, y + dy

        # Valid step move
        if grid[nx][ny] == 0 or grid[nx][ny] == 'D':
            successors.append(
                {
                    'location': (nx, ny),
                    'parent': current_node,
                    'cost': current_node['cost'] + 1
                }
            )

    # Slide movement
    for dx, dy in directions:
        nx, ny = x, y

        while grid[nx + dx][ny + dy] == 0:
            nx += dx
            ny += dy

        # Valid slide move
        if grid[nx][ny] == 'D' or (nx, ny) != (x, y):
            successors.append(
                {
                    'location': (nx, ny),
                    'parent': current_node,
                    'cost': current_node['cost'] + 1
                }
            )

    return successors

### 6) Extract plan from the final node

In [323]:
def extract_plan(current_node):
    plan = []
    while current_node:
        plan.append(current_node['location'])
        current_node = current_node['parent']
    return plan[::-1]  # Reverse the plan to get from start to goal

### 7) BFS function to find the plan

In [324]:
def find_plan_bfs(grid, start_loc, goal_loc):
    # Start state
    start_node = {
        'location': start_loc,
        'parent': None,
        'cost': 0
    }

    # Fringe
    fringe = deque([start_node])

    # Visited set to keep track of explored nodes
    visited = set()
    visited.add(start_loc)

    while fringe:
        current_node = fringe.popleft()
        current_loc = current_node['location']

        # Check if we've reached the goal
        if goal_test(current_loc, goal_loc):
            return extract_plan(current_node), current_node['cost']

        # Get the successors and add them to the fringe
        for successor in successor_fcn(grid, current_node):
            if successor['location'] not in visited:
                visited.add(successor['location'])
                fringe.append(successor)

    return "No path found", None

### 8) DFS function to find the plan

In [325]:
def find_plan_dfs(grid, start_loc, goal_loc):
    # Start state
    start_node = {
        'location': start_loc,
        'parent': None,
        'cost': 0
    }

    # Fringe
    fringe = [start_node]

    # Visited set to keep track of explored nodes
    visited = set()
    visited.add(start_loc)

    while fringe:
        current_node = fringe.pop()
        current_loc = current_node['location']

        # Check if we've reached the goal
        if goal_test(current_loc, goal_loc):
            return extract_plan(current_node), current_node['cost']

        # Get the successors and add them to the fringe
        for successor in successor_fcn(grid, current_node):
            if successor['location'] not in visited:
                visited.add(successor['location'])
                fringe.append(successor)

    return "No path found", None

### 9) Run Time!

In [326]:
# BFS
print('BFS')
plan_bfs, cost_bfs = find_plan_bfs(grid, start_loc, goal_loc)
print("Plan:", plan_bfs)
print("Cost:", cost_bfs)

# DFS
print('\nDFS')
plan_dfs, cost_dfs = find_plan_dfs(grid, start_loc, goal_loc)
print("Plan:", plan_dfs)
print("Cost:", cost_dfs)


BFS
Plan: [(6, 14), (5, 14), (5, 16), (4, 16)]
Cost: 3

DFS
Plan: [(6, 14), (6, 17), (8, 17), (8, 16), (5, 16), (4, 16)]
Cost: 5


### 10) Comparison and Analysis


In [327]:
if cost_bfs is None or cost_dfs is None:
    print("No path found. So we cannot summarize anything. Please try the different grid.")
else:
    best_algorithm = "BFS" if cost_bfs < cost_dfs else "DFS"
    best_cost = cost_bfs if cost_bfs < cost_dfs else cost_dfs

    worst_algorithm = "BFS" if cost_bfs > cost_dfs else "DFS"
    worst_cost = cost_bfs if cost_bfs > cost_dfs else cost_dfs

    is_equal = cost_bfs == cost_dfs

    print("To decide which one is more efficient, we can take a look at the costs.")

    if is_equal:
        print("They both (BFS & DFS) needed the same cost to reach destination.")
    else:
        print(f"As you can see, to reach destination, {best_algorithm} took less cost ({best_cost}) than {worst_algorithm} ({worst_cost}).")

To decide which one is more efficient, we can take a look at the costs.
As you can see, to reach destination, BFS took less cost (3) than DFS (5).
